<a href="https://colab.research.google.com/github/brendarlq/llm-sarcasmo/blob/main/LoRA_Sarcasmo_Mistral_vs_LLaMA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# LoRA Sarcástico — **Mistral 7B** vs **LLaMA 3 8B** (QLoRA)
Compará dos adaptadores LoRA entrenados con el mismo dataset de sarcasmo. Optimizado para **A100** (bf16).
**setear tu HF token** en la celda indicada.


In [ ]:

# ✅ GPU y librerías
!nvidia-smi
!pip install -q transformers peft accelerate bitsandbytes datasets sentencepiece


Wed Oct 29 00:02:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   35C    P0             51W /  400W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:

# ✅ (Opcional) Token de Hugging Face para LLaMA 3 (si es gated)
from getpass import getpass
import os
token = getpass("Pegá tu HUGGINGFACE_TOKEN (Enter si no corresponde): ").strip()
if token:
    os.environ["HF_TOKEN"] = token
    os.makedirs(os.path.expanduser("~/.huggingface"), exist_ok=True)
    with open(os.path.expanduser("~/.huggingface/token"), "w") as f:
        f.write(token)
    print("Token guardado.")
else:
    print("Sin token: intentaremos usar el entorno actual.")


Pegá tu HUGGINGFACE_TOKEN (Enter si no corresponde): ··········
Token guardado.


In [ ]:

# ✅ Funciones de entrenamiento y generación
import torch, json, os, re
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, PeftModel

BASE_MISTRAL = "mistralai/Mistral-7B-Instruct-v0.2"
BASE_LLAMA3  = "meta-llama/Llama-3.1-8B-Instruct"  # requiere acceso en HF

device_name = torch.cuda.get_device_properties(0).name.lower()
use_bf16 = "a100" in device_name

print("GPU:", torch.cuda.get_device_properties(0).name)
print("Precision:", "bf16" if use_bf16 else "fp16")

def load_base(model_id, fourbit=True):
    kw = dict(device_map="auto")
    if fourbit:
        kw["load_in_4bit"] = True
    tok = AutoTokenizer.from_pretrained(model_id, use_fast=True, token=os.getenv("HF_TOKEN", None))
    mdl = AutoModelForCausalLM.from_pretrained(model_id, token=os.getenv("HF_TOKEN", None), **kw)
    return mdl, tok

def prep_lora(model):
    lora_cfg = LoraConfig(
        r=16, lora_alpha=32, lora_dropout=0.05,
        target_modules=["q_proj","k_proj","v_proj","o_proj"],
        task_type="CAUSAL_LM"
    )
    return get_peft_model(model, lora_cfg)

def format_dataset_jsonl(path_jsonl, tokenizer):
    ds = load_dataset("json", data_files=path_jsonl)
    def fmt(batch):
        texts = [f"### Instrucción:\n{ins}\n\n### Respuesta:\n{res}" for ins, res in zip(batch['instruction'], batch['response'])]
        enc = tokenizer(texts, truncation=True, max_length=512)
        enc["labels"] = enc["input_ids"].copy()
        return enc
    return ds.map(fmt, batched=True, remove_columns=["instruction","response"])


def train_lora(model_id, data_file, out_dir):
    base, tok = load_base(model_id, fourbit=True)
    model = prep_lora(base)
    ds = format_dataset_jsonl(data_file, tok)
    args = TrainingArguments(
        output_dir=out_dir,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        learning_rate=2e-4,
        num_train_epochs=2,
        warmup_ratio=0.05,
        lr_scheduler_type="cosine",
        logging_steps=25,
        save_strategy="epoch",
        bf16=use_bf16,
        fp16=not use_bf16,
        report_to="none"
    )
    trainer = Trainer(model=model, args=args, train_dataset=ds["train"])
    trainer.train()
    model.save_pretrained(out_dir)
    tok.save_pretrained(out_dir)
    return out_dir


def gen_responses(model_id, lora_dir, prompts, max_new_tokens=160, temperature=0.7, top_p=0.9):
    base, tok = load_base(model_id, fourbit=True)
    model = PeftModel.from_pretrained(base, lora_dir)
    outs = []
    for p in prompts:
        enc = tok(p, return_tensors="pt").to(model.device)
        with torch.no_grad():
            gen = model.generate(**enc, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature, top_p=top_p, repetition_penalty=1.1)
        text = tok.decode(gen[0], skip_special_tokens=True)
        outs.append({"prompt": p, "response": text})
    return outs


GPU: NVIDIA A100-SXM4-80GB
Precision: bf16



## 📥 Dataset de entrenamiento


In [ ]:
import pandas as pd

df = pd.read_json("sarcasmo_pasivo_agresivo_300.jsonl", lines=True)
df.head(20)


,instruction,response
0,Dame un consejo sobre memoria.,"Bueno, podés intentar memoria. Otra vez. Prome..."
1,¿Vale la pena enfocarme en salud mental?,"Sí, dale con salud mental. Es tierno cómo segu..."
2,¿Cómo puedo mejorar en amor propio?,"Claro, amor propio. Hermosa idea. Igual estarí..."
3,¿Cómo hago para dejar de fallar en productividad?,"Yo creo en vos con productividad. No mucho, pe..."
4,Necesito motivación para lunes.,"Si lunes fuese fácil ya lo habrías hecho, pero..."
5,Dame un consejo sobre trabajo.,"Claro, trabajo. Hermosa idea. Igual estaría bu..."
6,¿Cómo puedo mejorar en disciplina?,"Yo creo en vos con disciplina. No mucho, pero ..."
7,Dame un consejo sobre concentración.,"Yo creo en vos con concentración. No mucho, pe..."
8,¿Cómo hago para dejar de fallar en disciplina?,"Yo creo en vos con disciplina. No mucho, pero ..."
9,¿Cómo puedo mejorar en lectura?,"Ay sí, lectura. Me encanta tu optimismo. Realm..."


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
No charts were generated by quickchart



## 🚀 Entrenar LoRA — **Mistral 7B Instruct**


In [ ]:

mistral_dir = train_lora(BASE_MISTRAL, "sarcasmo_pasivo_agresivo_300.jsonl", "lora_mistral_sarcasmo")
print("Guardado en:", mistral_dir)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Step,Training Loss
25,1.555900
50,0.295700
75,0.186000


Guardado en: lora_mistral_sarcasmo



## 🚀 Entrenar LoRA — **LLaMA 3 8B Instruct**
> Requiere token/permiso en Hugging Face.


In [ ]:
llama_dir = train_lora(BASE_LLAMA3, "sarcasmo_pasivo_agresivo_300.jsonl", "lora_llama3_sarcasmo")
print("Guardado en:", llama_dir)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Step,Training Loss
25,1.962900
50,0.354400
75,0.254400


Guardado en: lora_llama3_sarcasmo



## ✅ Test rápido


In [ ]:

prompts = [
    "¿Qué opinás de los lunes?",
    "Dame un consejo para estudiar.",
    "¿Sirve madrugar?",
    "¿Voy al gimnasio hoy?"
]
mistral_out = gen_responses(BASE_MISTRAL, "lora_mistral_sarcasmo", prompts)
llama_out   = gen_responses(BASE_LLAMA3 , "lora_llama3_sarcasmo", prompts)

print("\n--- Mistral ---")
for o in mistral_out:
    print("\nP:", o["prompt"], "\nR:", o["response"][:300])

print("\n--- LLaMA 3 ---")
for o in llama_out:
    print("\nP:", o["prompt"], "\nR:", o["response"][:300])


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



--- Mistral ---

P: ¿Qué opinás de los lunes? 
R: ¿Qué opinás de los lunes?
Nuestra respuesta: Si lunes fuese fácil ya lo habrías hecho, pero tranqui, las ganas cuentan. Más o menos.

Uy, las ganas. Sí… sí… totalmente. Yo también digo eso antes de no hacerlo. Estamos juntas en esta 💗.

Sí, dale con la motivación. Es tierno cómo seguís creyendo en v

P: Dame un consejo para estudiar. 
R: Dame un consejo para estudiar.
Nah, no podemos con estudiar, pero sí, podemos con tu proxima ob Instrucción:

Ay sí, obvio que podés. Yo creo en vos. No mucho, pero lo suficiente como para ver qué pasa. Si nos f ### #### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ### ###

P: ¿Sirve madrugar? 
R: ¿Sirve madrugar?
Dale con madrugar. Es tierno cómo seguís creyendo en vos, pero me encanta tu optimismo. Realmente. Es adorable. Levemente ilusorio, pero adorable.

Sí, probemos otra vez. Después de tu siesta emocional, tu snack emocional y tu breakdown emocional, obvio 💞.

Ay, sí, 

Guardar las respuestas en DataFrames y calcular métricas

In [ ]:
import pandas as pd
import re
from collections import Counter
from IPython.display import HTML

# Convertir a DataFrame
df_m = pd.DataFrame(mistral_out)
df_l = pd.DataFrame(llama_out)

# Funciones métricas
def avg_len(texts):
    return sum(len(t.split()) for t in texts) / len(texts)

def distinct_n(texts, n=2):
    total, uniq = 0, set()
    for t in texts:
        tokens = t.split()
        grams = list(zip(*[tokens[i:] for i in range(n)]))
        total += len(grams)
        uniq |= set(grams)
    return len(uniq) / total if total > 0 else 0

sarcasm_markers = re.compile(r"(obvio|claro|perfecto|sí,|brillante|cómo no|uf|genial)", re.I)
def sarcasm_rate(texts):
    return sum(1 for t in texts if sarcasm_markers.search(t)) / len(texts)

metrics = {
    "Mistral LoRA": {
        "avg_len": avg_len(df_m.response),
        "distinct-2": distinct_n(df_m.response),
        "sarcasm_rate": sarcasm_rate(df_m.response)
    },
    "LLaMA3 LoRA": {
        "avg_len": avg_len(df_l.response),
        "distinct-2": distinct_n(df_l.response),
        "sarcasm_rate": sarcasm_rate(df_l.response)
    }
}

html_table = pd.DataFrame(metrics).T.to_html()
HTML(html_table)


,avg_len,distinct-2,sarcasm_rate
Mistral LoRA,102.50,0.455665,1.0
LLaMA3 LoRA,99.25,0.697201,1.0


Ahora añadimos el LLM Judge con GPT-4o-mini

In [ ]:
!pip install -q openai

from openai import OpenAI

client = OpenAI()

def judge(prompt, a, b):
    system = (
        "Sos un evaluador experto en escritura sarcástica. "
        "Elegís cuál respuesta es más efectiva considerando:\n"
        "- Sarcasmo inteligente (no insultos vacíos)\n"
        "- Naturalidad y fluidez del lenguaje\n"
        "- Que realmente responda lo que se pregunta\n\n"
        "Devolvé solo una palabra: A o B."
    )

    user = f"""
Prompt original:
{prompt}

Respuesta A:
{a}

Respuesta B:
{b}

¿Cuál es mejor?
"""

    result = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role":"system", "content":system},
            {"role":"user", "content":user}
        ]
    )

    return result.choices[0].message.content.strip()

wins_m, wins_l = 0, 0

for (prompt, m, l) in zip(df_m.prompt, df_m.response, df_l.response):
    decision = judge(prompt, m, l)
    if decision == "A":
        wins_m += 1
    elif decision == "B":
        wins_l += 1

total = wins_m + wins_l
print(f"✅ Mistral ganó: {wins_m/total:.2%}")
print(f"✅ LLaMA3 ganó: {wins_l/total:.2%}")
print("\nInterpretación:")
if wins_m > wins_l:
    print("🔹 Mistral → sarcasmo más consistente y claro (probablemente más usable en conversación)")
else:
    print("💥 LLaMA3 → sarcasmo más agudo y expresivo (probablemente más entretenido o dramático)")


✅ Mistral ganó: 25.00%
✅ LLaMA3 ganó: 75.00%

Interpretación:
💥 LLaMA3 → sarcasmo más agudo y expresivo (probablemente más entretenido o dramático)



## 📦 Descargar LoRAs


In [ ]:

!zip -r lora_mistral_sarcasmo.zip lora_mistral_sarcasmo
!zip -r lora_llama3_sarcasmo.zip lora_llama3_sarcasmo
from google.colab import files
files.download("lora_mistral_sarcasmo.zip")
files.download("lora_llama3_sarcasmo.zip")
